In [1]:
import os
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
import torch
from sentence_transformers import SentenceTransformer
from torch.nn import functional as F
import numpy as np

/home/niloy/miniconda3/envs/pdf_rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pymilvus import (FieldSchema, DataType, CollectionSchema, Collection)

In [4]:
from pymilvus import connections,MilvusClient

In [5]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory, CombinedMemory
from langchain.chains import LLMChain
from langchain_core.documents.base import Document

In [6]:
import time
# from transformers import AutoTokenizer, pipeline, AutoConfig, AutoModelForCausalLM, AutoModel
from tqdm import tqdm
import json
from langchain_community.llms import CTransformers
import tiktoken

In [7]:
import fitz
import re, glob

In [ ]:
# loader = DirectoryLoader(
#     os.path.abspath("/home/niloy/code/simple_chat/persist_storage"),
#     glob="**/*.pdf",
#     use_multithreading=True,
#     show_progress=True,
#     max_concurrency=50,
#     loader_cls=UnstructuredPDFLoader,
# )
# documents = loader.load()

In [ ]:
# toc_regex = r"""
#         ^(?:(?:Table of )?Contents|List of (?:Figures|Tables))?  # Optional Titles
#         (?:\s*\n+)?                                             # Optional newline(s) 
#         (?:\s*\d+\.?\s+(.+)\s*)?                                # Optional: number, period, title 
#         \s*\.{2,}\s*\d+                                         # Required dotted leader line and page number
#     """

# pdf_contents = []
# for page in tqdm(temp_data):
#     page_data = page.page_content
#     if (flags & flag_content_started) == flag_content_started:
#         pdf_contents += page
#     else:
#         llm_answer = llm_tocqa_chain.invoke({"context": page_data})
#         print(page_data)
#         print(llm_answer["text"])
#         if llm_answer["text"].lower().startswith("no"):
#             pdf_contents += page
#         else:
#             flags = flags | flag_toc_found
#         if llm_answer["text"].lower().startswith("no") and (flags == flag_toc_found):
#             flags = flags | flag_toc_ended


In [8]:
def extract_page_contents(pdf_path):
    toc_dotted_line_regex = r".*\.{2,}\s*\d+"
    pdf_name = os.path.basename(pdf_path)
    print("saving .. " + pdf_name)
    original_doc = fitz.open(pdf_path)
    output_doc = fitz.open()
    pages_to_exclude = [] 
    flag_toc_found = 1
    flag_toc_ended = 2
    flag_content_started = 3
    flags = 0
    for i, page in enumerate(original_doc):
        text = page.get_text()
        if (flags & flag_content_started) == flag_content_started:
            break
        else:
            if len(re.findall(toc_dotted_line_regex, text)) > 0:
                pages_to_exclude.append(i)
                flags = flags | flag_toc_found
            elif (flags & flag_toc_found) == flag_toc_found and len(re.findall(toc_dotted_line_regex, text)) == 0:
                flags = flags | flag_toc_ended
    output_doc = original_doc
    print("excluding pages ... " + str(pages_to_exclude))
    output_doc.delete_pages(from_page=1, to_page=pages_to_exclude[-1])
    output_doc.save(pdf_name)

    temp_loader = PyMuPDFLoader(pdf_name)
    temp_data = temp_loader.load()
    os.remove(pdf_name)
    
    return temp_data
    

In [9]:
def load_from_directory(directory_path):
    pdfs = glob.glob(directory_path + "/*.pdf")
    documents = []
    for pdf in tqdm(pdfs):
        documents.extend(extract_page_contents(pdf))
    return documents

In [11]:
documents = load_from_directory("persist_storage")

 14%|█████████▊                                                           | 1/7 [00:00<00:00,  6.45it/s]

saving .. 3EST000235-2562_Ben_CCU-S2_Safety_Manual.pdf
excluding pages ... [1, 2]
saving .. 3EST000235-2511_Ben_CCU-S2_ECP-USB_Content_Specification.pdf
excluding pages ... [1, 2, 3]


 43%|█████████████████████████████▌                                       | 3/7 [00:00<00:00,  5.81it/s]

saving .. 3EST000236-8112_Ben_CCU-S2_Product_Release Note_1.2.1.0.pdf
excluding pages ... [1, 2, 3]
saving .. 3EGM081750-0035_Gen_TBCI_and_TBI-ED_Interface_Control_Document.pdf
excluding pages ... [1, 2, 3, 4, 5]


 57%|███████████████████████████████████████▍                             | 4/7 [00:00<00:00,  3.85it/s]

saving .. 3EST000235-2556_Een_CCU-S2_User_Manual.pdf
excluding pages ... [5, 6, 7, 8, 9]


 71%|█████████████████████████████████████████████████▎                   | 5/7 [00:01<00:00,  2.42it/s]

saving .. 3EGM007200D2806_Yen_MITRAC_Control_C&C_Platform_Catalogue.pdf
excluding pages ... [1, 2, 3, 4, 5, 6]


100%|█████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  2.77it/s]

saving .. 3EST000235-2552_Aen_CCU-S2_Interface_Control_Document.pdf
excluding pages ... [1, 2]


In [12]:
len(documents)

596

In [13]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
docs = splitter.split_documents(documents)

In [ ]:
# Initialize torch settings
DEVICE = torch.device('cuda:0' 
   if torch.cuda.is_available() 
   else 'cpu')
# Load the encoder model from huggingface model hub.
model_name = "mixedbread-ai/mxbai-embed-large-v1"
encoder = SentenceTransformer(model_name, device=DEVICE)


# Get the model parameters and save for later.
MAX_SEQ_LENGTH = encoder.get_max_seq_length() 
EMBEDDING_LENGTH = encoder.get_sentence_embedding_dimension()


In [21]:
chunk_list = []
for chunk in tqdm(docs):
    # Generate embeddings using encoder from HuggingFace.
    embeddings = torch.tensor(encoder.encode([chunk.page_content]))
    embeddings = F.normalize(embeddings, p=2, dim=1)
    converted_values = list(map(np.float32, embeddings))[0]
    # Assemble embedding vector, original text chunk, metadata.
    chunk_dict = {
        'vector': converted_values,
        'text': chunk.page_content,
        'source': chunk.metadata['source'],}
    chunk_list.append(chunk_dict)

100%|███████████████████████████████████████████████████████████████| 1344/1344 [01:04<00:00, 20.77it/s]


In [10]:
encoding = tiktoken.get_encoding("cl100k_base")

In [16]:
ENDPOINT=f'http://localhost:19530'
client = MilvusClient(
  uri=ENDPOINT)

In [17]:
EMBEDDING_LENGTH

1024

In [18]:
COLLECTION_NAME = "MilvusDocs"

In [26]:
client.drop_collection(collection_name=COLLECTION_NAME)

In [27]:
# 1. Define a minimum expandable schema.
fields = [
   FieldSchema("pk", DataType.INT64, is_primary=True, auto_id=True),
   FieldSchema("vector", DataType.FLOAT_VECTOR, dim=EMBEDDING_LENGTH),]
schema = CollectionSchema(
   fields,
   enable_dynamic_field=True,)

index_params = client.prepare_index_params()

index_params.add_index(
    field_name="vector",
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

client.create_collection(collection_name=COLLECTION_NAME, schema=schema, index_params=index_params)

# 2. Create the collection.
# mc = Collection("MilvusDocs", schema)

# 3. Index the collection.
# mc.create_index(
#    field_name="vector",
#    index_params={
#        "index_type": "AUTOINDEX",
#        "metric_type": "COSINE",})

In [28]:
# Insert data into the Milvus collection.
# insert_result = mc.insert(chunk_list)
insert_result = client.insert(collection_name=COLLECTION_NAME, data=chunk_list)

# After final entity is inserted, call flush 
# to stop growing segments left in memory.

# print(mc.partitions)

In [19]:
client.describe_collection(collection_name=COLLECTION_NAME)

{'collection_name': 'MilvusDocs',
 'auto_id': True,
 'num_shards': 1,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'pk',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'vector',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 1024}}],
 'aliases': [],
 'collection_id': 449480778103619488,
 'consistency_level': 2,
 'properties': {},
 'num_partitions': 1,
 'enable_dynamic_field': True}

In [20]:
res = client.query(collection_name=COLLECTION_NAME, output_fields=["count(*)"])
res

[{'count(*)': 1344}]

In [21]:
# template = """

# Context: {context}

# Question: {question}

# Answer: Let's work this out in a step by step way to be sure we have the right answer."""

template = """
[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.  Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Previous Conversation:
{chat_history}

Context: {context}

Question: {question}

[/INST]

"""

summary_template = """
[INST] <<SYS>>
As a professional summarizer, create a concise and comprehensive summary of the provided context, while adhering to these guidelines:
* Craft a summary that is detailed, thorough, in-depth, and complex, while maintaining clarity and conciseness.
* Incorporate main ideas and essential information, eliminating extraneous language and focusing on critical aspects.
* Rely strictly on the provided text, without including external information.
* Format the summary in paragraph form for easy understanding.
<</SYS>>

Context: {context}

[/INST]

"""

toc_qa_prompt = """
[INST] <<SYS>>
Here are characteristics of table of contents pages and main content pages:
Table of Contents: Has numbered list, Lists chapter/section titles, lists tables/figures, has dots leading to numbers, often titled as "table of contents"/ "list of figures"/ "list of tables".
Main Content: Contains names of sections and the description of those sections, paragraphs of text, likely longer.

Considering these features, is the following page_content a table of contents page? Answer only in yes or no.
<</SYS>>

page_content: {context}

[/INST]

"""

prompt = PromptTemplate.from_template(template)
summary_prompt = PromptTemplate.from_template(summary_template)
toc_qa_prompt = PromptTemplate.from_template(toc_qa_prompt)

In [22]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [17]:
torch.cuda.empty_cache()

In [23]:
n_gpu_layers = 24  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 8  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
n_ctx = 4096

# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GGUF", model_file="llama-2-7b-chat.q4_K_M.gguf", model_type="llama", gpu_layers=50)


# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="models/llama-2-7b-chat.Q4_K_M.gguf",
    n_ctx=n_ctx,
    n_gpu_layers = n_gpu_layers,
    n_batch=n_batch,
    # callback_manager=callback_manager,
    # verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32         

In [24]:
QUESTION = "What are the complex type tables?"
QUERY = [QUESTION]

# Before conducting a search, load the data into memory.
client.load_collection(collection_name=COLLECTION_NAME)

# Embed the question using the same encoder.
embedded_question = torch.tensor(encoder.encode([QUESTION]))
# Normalize embeddings to unit length.
embedded_question = F.normalize(embedded_question, p=2, dim=1)
# Convert the embeddings to list of list of np.float32.
embedded_question = list(map(np.float32, embedded_question))

# Return top k results with AUTOINDEX.
TOP_K = 5

# Run semantic vector search using your query and the vector database.
start_time = time.time()
results = client.search(
    collection_name=COLLECTION_NAME,
    data=embedded_question, 
    anns_field="vector", 
    output_fields=["text", "source"], 
    limit=TOP_K)

elapsed_time = time.time() - start_time
print(f"Milvus search time: {elapsed_time} sec")


Milvus search time: 0.0044934749603271484 sec


In [ ]:
for n, hits in enumerate(results):
    print(f"{n}th query result")
    for hit in hits:
        print(hit)


In [33]:
# Assemble the context as a stuffed string.
context = ""
for r in results[0]:
    text = r['entity']['text']
    context += f"{text} "

# Also save the context metadata to retrieve along with the answer.
context_metadata = {
   "source": results[0][0]["entity"]["source"],}

In [ ]:
context

In [ ]:
context_metadata

In [ ]:
full_question = prompt.format(context = context, question = question)
full_question

In [24]:
chat_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1000, memory_key="chat_history", input_key="question")
# context_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit = 2000, memory_key = "context", input_key="question")
# memory = CombinedMemory(memories=[chat_memory, context_memory])

In [25]:
summary_token_limit = 2000
llm_chain = LLMChain(llm=llm, prompt=prompt, memory=chat_memory, verbose=True)
llm_summary_chain = LLMChain(llm = llm, prompt=summary_prompt)
llm_tocqa_chain = LLMChain(llm=llm, prompt=toc_qa_prompt)

llm_chain = prompt | llm
llm_summary_chain = summary_prompt | llm

In [26]:
def check_token_count(context, encoding):
    encoded_question = encoding.encode(context)
    print("token count: ")
    print(len(encoded_question) + 70)
    return len(encoded_question) + 70

In [27]:
def get_context(results):
    context = ""
    for r in results[0]:
        text = r['entity']['text']
        context += f"{text} "
    
    # Also save the context metadata to retrieve along with the answer.
    context_metadata = {
       "source": results[0][0]["entity"]["source"],}
    return context

In [28]:
def generate_context(question, milvus_client, COLLECTION_NAME, encoder, context, encoding, prompt):
    
    milvus_client.load_collection(collection_name=COLLECTION_NAME)
    # Embed the question using the same encoder.
    embedded_question = torch.tensor(encoder.encode([question]))
    # Normalize embeddings to unit length.
    embedded_question = F.normalize(embedded_question, p=2, dim=1)
    # Convert the embeddings to list of list of np.float32.
    embedded_question = list(map(np.float32, embedded_question))
    
    # Return top k results with AUTOINDEX.
    TOP_K = 5
    
    # Run semantic vector search using your query and the vector database.
    start_time = time.time()
    results = milvus_client.search(
        collection_name=COLLECTION_NAME,
        data=embedded_question, 
        anns_field="vector", 
        output_fields=["text", "source"], 
        limit=TOP_K)
    
    elapsed_time = time.time() - start_time
    print(f"Milvus search time: {elapsed_time} sec")
    new_context = get_context(results)
    temp_context = context + f"{new_context}"
    # full_question = prompt.format(context = temp_context, question = question)
    if check_token_count(temp_context, encoding) > summary_token_limit:
        result = llm_summary_chain.invoke({"context": temp_context})
        return result['text']
    else:
        return temp_context
    

In [ ]:
context = ""
chat_memory.clear()
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Thanks!")
        break
    # context = ""
    context = generate_context(user_input, client, COLLECTION_NAME, encoder, context, encoding, prompt)
    result = llm_chain.invoke({"context": context, "question": user_input})
    response = result['text']
    print("Chatbot:", response)

You:  what is ccu s2?


Milvus search time: 0.01741504669189453 sec
token count: 
1082


> Entering new LLMChain chain...
Prompt after formatting:

[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.  Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Previous Conversation:


Context: CR no. 
Description 
NA 
This is the first release of the CCU-S2 System Software. Product name: 
CCU-S2 
 
Version: 
1.2.0 
 
Doc. type: User Manual 
 
Doc ID-number: 3EST000235-2556 
 
Revision: _E 
 
Langauge: en  
Page: 24/191 
 
 
 
© ALSTOM SA 2022. All rights reserved. Reproduction, use or disclosure to third parties, without express written authorization from Alstom,  
is strictly prohibited. 
3 Update factory installed unit 
T